# Start:
Please read the doc first: https://slack-transfer.readthedocs.io/en/latest/user_guide/run.html

まずはドキュメントを読んで全体像を把握してください: https://slack-transfer.readthedocs.io/en/latest/user_guide/run_ja.html


# Install slack_transfer

In [ ]:
!pip install slack-transfer

# import necessary packses

In [ ]:
from google.colab import drive
import slack_transfer
import os
import math

# mount google drive

In [ ]:
drive.mount('/content/drive')

# Set local data directory
 - "/content/drive/MyDrive/" is correspinding to my drive (マイドライブ)
 - "/content/drive/Shareddrives/" is correspinding to share drive (共有ドライブ)

In [ ]:
local_data_dir = "/content/drive/................../"  
_local_data_dir_parent = os.path.dirname(os.path.abspath(local_data_dir))

check the parent folder exists or not． (local_data_dir will be automatically generated, but the parent directory have to exist for safe operation)

親ディレクトリが存在するか確認します． (local_data_dirは自動生成されますが，安全のためその親ディレクトリは存在している必要があります．)

In [ ]:
ls $_local_data_dir_parent

See:
 - https://slack-transfer.readthedocs.io/en/latest/user_guide/run.html#downloader-token
 - https://slack-transfer.readthedocs.io/en/latest/user_guide/run.html#uploader-token

以下を参考にtokenを取得し，入力してください:
 - https://slack-transfer.readthedocs.io/en/latest/user_guide/run_ja.html#downloader-token-ja
 - https://slack-transfer.readthedocs.io/en/latest/user_guide/run_ja.html#uploader-token-ja

In [ ]:
downloader_token="xoxb-...................."
uploader_token="xoxb-...................."

Invite the bot on the original WS to private channels if necessary.
See:
 - https://slack-transfer.readthedocs.io/en/latest/user_guide/run.html#invite-private

必要に応じて，private channelにAPI botを追加してください．参考:
 - https://slack-transfer.readthedocs.io/en/latest/user_guide/run_ja.html#invite-private-ja

# Test Slack tokenのテスト

In [ ]:
client = slack_transfer._base.CommonNoLocalVolumeClient(token=downloader_token)
client.test_connection()
client.test_downloader()
client = slack_transfer._base.CommonNoLocalVolumeClient(token=uploader_token)
client.test_connection()
client.test_uploader()

# Check the volume required on google drive
Google drive上に十分な容量があるか確認してください．

In [ ]:
client = slack_transfer._base.CommonNoLocalVolumeClient(token=downloader_token)
volume = client.get_file_volumes(channel_ids=None)
units = ["B", "KB", "MB", "GB", "TB", "PB", "EB", "ZB"]
i_units = math.floor(math.log(volume, 1024)) if volume > 0 else 0
size = round(volume / 1024**i_units, 2)
print(f"{size} {units[i_units]} (Actual download size is usually smaller.)")

# Download Emojis

In [ ]:
downloader = slack_transfer.DownloaderClient(local_data_dir=local_data_dir, token=downloader_token)
downloader.download_emoji()

Emojis are downloaded to the filder named emojis in the folder you set beforehand. Please migrate emojis to the destination WS. See also: https://slack-transfer.readthedocs.io/en/latest/user_guide/run.html#migrate-emojis

指定したディレクトリのemojisフォルダーに絵文字がダウンロードされています．必要に応じて，Upload側WSにインポートしてください．参考: https://slack-transfer.readthedocs.io/en/latest/user_guide/run_ja.html#migrate-emojis-ja

# Run migration

Next, let's migration. If you have already downloaded the data and it failed during upload, set skip_download=True when name_mappings is necessary. Please refer to https://slack-transfer.readthedocs.io/en/latest/user_guide/run.html#channel-mappings. If needed, please set as dictionary like `{"general":"_general", "random":"_random"}`.

続いて，移行を行います．すでにデータダウンロード済みで，アップロード中に失敗した場合などは，skip_download=Trueを設定してください．また，name_mappingsは https://slack-transfer.readthedocs.io/en/latest/user_guide/run_ja.html#channel-mappings-ja を参考にして，必要に応じて設定してください．`{"general":"_general", "random": "_random"}`などとdictonary型で設定します．

In [ ]:
slack_transfer.run(
    downloader_token=downloader_token,
    uploader_token=uploader_token,
    override=False,
    skip_download=False,
    local_data_dir=local_data_dir,
    name_mappings={},
    channel_names=[],
)

If the process fails: Check the cause of the error. In many cases, the cause may be a potential library bug, but if the process stops due to a broken session of this notebook, it may be possible to restart by deleting the channel that failed in the destination side WS and trying again.

処理に失敗した場合：
エラーが出るので，原因を確認します．多くの場合，潜在的なライブラリのバグの可能性がありますが，セッション切れで途中で止まってしまった場合は，Upload側WSで処理が失敗したチャンネルを丸々Deleteして再度行うとうまく行く場合があります．